In [2]:
import pandas as pd
from textblob import TextBlob

# Language Detection

In [5]:
# !pip install langdetect
# !pip install langid

## TextBlob
https://towardsdatascience.com/4-python-libraries-to-detect-english-and-non-english-language-c82ad3efd430

In [3]:
reports = pd.read_csv('../data_ptb-xl/train.csv')

In [3]:
lang = TextBlob(reports['preprocessed_report'][0])
language = lang.detect_language()

AttributeError: 'TextBlob' object has no attribute 'detect_language'

## langdetect

In [6]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def detect_language_langdetect(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

In [7]:
reports['langdetect'] = reports['preprocessed_report'].apply(detect_language_langdetect)

In [8]:
reports['langdetect'].value_counts()

langdetect
en    11691
cy      931
es      212
ro       51
sv       41
ca       28
da       24
nl       18
no       17
fr       17
it       16
fi       15
de       15
et        6
lt        5
sq        3
cs        2
pt        1
sl        1
id        1
Name: count, dtype: int64

## langid

In [4]:
import langid

def detect_language_langid(text):
    try:
        language, _ = langid.classify(text)
        return language
    except LangDetectException:
        return "unknown"

In [5]:
reports['langid'] = reports['preprocessed_report'].apply(detect_language_langid)

In [9]:
reports['langid'].value_counts()

langid
en    10337
lt     1581
fr      515
da      158
no       94
es       64
et       42
de       37
it       34
sl       28
la       22
cy       22
nl       21
oc       19
ca       18
ro       17
sv       13
eo       12
br       12
sk       11
id        9
af        5
se        5
lb        4
qu        3
pl        2
wa        2
nb        2
nn        2
fi        1
tl        1
ht        1
cs        1
Name: count, dtype: int64

In [13]:
reports[['ecg_id', 'report']].to_csv('ptb-xl_report.csv', index=False)

todo:
https://aws.amazon.com/blogs/industries/how-to-process-medical-text-in-multiple-languages-using-amazon-translate-and-amazon-comprehend-medical/
https://arxiv.org/pdf/2311.16588.pdf